## Making the trianable ansatz

In [9]:
import numpy as np

I = np.eye(2)
n_qubits = 6
size_of_vec = 2**n_qubits
num_layers = 5
extraStates=True

from utils import DFS
inputStates, expectedStates = DFS().getInitialTargetStates(extraStates=extraStates)
modifiedInputStates, modifiedExpectedStates = DFS().getModifiedInitialTargetStates(extraStates=extraStates)

In [10]:
print(len(inputStates), len(expectedStates))
print(len(modifiedInputStates), len(modifiedExpectedStates))

12 12
12 12


In [11]:
def withinBounds(num):
    return num <= 1+1e-7 and num >= 1-1e-7
print('ensuring NON-MODIFIED states are normalized (aka <s|s> ~ 1)')
allStatesNormalized = True
for s in inputStates:
    dot = np.dot(s,s)
    if not withinBounds(dot):
        allStatesNormalized = False
print("All states are normalized" if allStatesNormalized else "All states are - NOT - normalized")

total = 0
for i in range(len(inputStates)):
    for j in range(len(inputStates)):
        if i != j:
            temp = np.dot(inputStates[i],inputStates[j])
            # print(f'<state{i+1}|state{j+1}> = {temp}')
            total += temp

orthoStatement = f'All {len(inputStates)} states are orthogonal with each other' if total <= 1e-14 else f'All {len(inputStates)} states are - NOT - orthogonal with each other'
print(orthoStatement)


print('-----------------------')

print('ensuring MODIFIED states are normalized (aka <s|s> ~ 1)')
allStatesNormalized = True
for s in inputStates:
    dot = np.dot(s,s)
    if not withinBounds(dot):
        allStatesNormalized = False
print("All states are normalized" if allStatesNormalized else "All states are - NOT - normalized")

total = 0
for i in range(len(modifiedInputStates)):
    for j in range(len(modifiedInputStates)):
        if i != j:
            temp = np.dot(modifiedInputStates[i],modifiedInputStates[j])
            # print(f'<state{i+1}|state{j+1}> = {temp}')
            total += temp

orthoStatement = f'All {len(inputStates)} states are orthogonal with each other' if total <= 1e-14 else f'All {len(inputStates)} states are - NOT - orthogonal with each other'
print(orthoStatement)

ensuring NON-MODIFIED states are normalized (aka <s|s> ~ 1)
All states are normalized
All 12 states are orthogonal with each other
-----------------------
ensuring MODIFIED states are normalized (aka <s|s> ~ 1)
All states are normalized
All 12 states are orthogonal with each other


In [29]:
def getLossFunctionOf4States(y_true, y_pred):
    from utils import MatrixUtils
    matrixUtils = MatrixUtils()
    loss = 0
    for i in range(len(y_true)):
        fidelity = matrixUtils.fidelity_statevector(y_true[i], y_pred[i])
        loss += fidelity
    return np.sqrt(1 - (1/4)*abs(loss))

In [25]:
def get_optimal_half_matrix(num_layers, addHalf=False, modified=False, pi_range=False, extraStates=False, batchOf4=None):
    import numpy as np
    from scipy.optimize import minimize
    from utils import MatrixUtils
    from utils import DFS
    
    if batchOf4 != None:
        extraStates = False

    originalInputStates, originalExpectedStates = DFS().getInitialTargetStates(extraStates=extraStates)
    modifiedInputStates, modifiedExpectedStates = DFS().getModifiedInitialTargetStates(extraStates=extraStates)

    if batchOf4 == 1:
        originalInputStates, originalExpectedStates = originalInputStates[:4], originalExpectedStates[:4]
        modifiedInputStates, modifiedExpectedStates = modifiedInputStates[:4], modifiedExpectedStates[:4]
    elif batchOf4 == 2:
        originalInputStates, originalExpectedStates = originalInputStates[4:], originalExpectedStates[4:]
        modifiedInputStates, modifiedExpectedStates = modifiedInputStates[4:], modifiedExpectedStates[4:]
    
    inputStates, expectedStates = (modifiedInputStates, modifiedExpectedStates) if modified else (originalInputStates, originalExpectedStates)
    
    matrixUtils = MatrixUtils(pi_range=pi_range, extended_states=extraStates)

    loss_function = matrixUtils.f_cnot_loss if batchOf4==None else getLossFunctionOf4States

    # Define the correct operations you want the matrix to perform on basis vectors
    def target_operations(parameters, inputStates, target_result):
        # Reshape the parameters into the matrix form
        normalParams = parameters[:num_layers*5]
        halfParams = parameters[num_layers*5:] if addHalf else None

        parameters = np.reshape(normalParams, (num_layers, 5))
        matrix = matrixUtils.get_total_matrix(size_of_vec=2**6, weights=parameters, halfWeights=halfParams)

        # Perform matrix multiplication with basis vectors
        results = []
        for i in range(len(inputStates)):
            results.append(np.matmul(matrix, inputStates[i]))

        # Calculate the loss as the difference between the obtained result and the target result
        # loss = square_loss(target_result, results)
        loss = loss_function(target_result, results)
        return loss

    # Generate random basis vectors and target result
    basis_vectors = np.array(modifiedInputStates) if modified else np.array(inputStates)
    target_result = np.array(modifiedExpectedStates) if modified else np.array(expectedStates)

    # Flatten the matrix parameters for optimization
    initial_parameters = np.ndarray.flatten(matrixUtils.get_random_weights(num_layers))

    initial_parameters = np.concatenate((initial_parameters, matrixUtils.get_random_half_layer_weights())) if addHalf else initial_parameters

    scale = np.pi if pi_range else 1
    bounds = [(-1*scale, 1*scale)] * len(initial_parameters)

    # Use scipy's minimize function to optimize the parameters
    result = minimize(target_operations, initial_parameters, args=(basis_vectors,target_result), method='L-BFGS-B', bounds=bounds)

    firstHalf, secondHalf = result.x[:num_layers*5], result.x[num_layers*5:] if addHalf else None
    # Reshape the optimized parameters back into the matrix form
    optimized_matrix = matrixUtils.get_total_matrix(size_of_vec=2**6, weights=firstHalf.reshape((num_layers, 5)), halfWeights=secondHalf)

    total_layers = num_layers + 0.5 if addHalf else num_layers

    # predStates = [np.matmul(optimized_matrix, mat) for mat in inputStates]
    predStates = matrixUtils.get_predictions(basis_vectors, weights=firstHalf.reshape((num_layers, 5)), halfWeights=secondHalf)
    fcnot_loss = loss_function(expectedStates, predStates)
    # print(f"f_cnot_loss for {total_layers} layers = {fcnot_loss}")
    # print(f"square_loss for {total_layers} layers = {matrixUtils.square_loss(expectedStates, predStates)}")
    return optimized_matrix, result.x, fcnot_loss

In [13]:
l = 13
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=False, extraStates=True)
print(f"Trained model (NON-MODIFIED) with extra states for params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=True, extraStates=True)
print(f"Trained model (MODIFIED) with extra states for params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=False, pi_range=True, extraStates=True)
print(f"Trained model (NON-MODIFIED) with extra states for params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=True, pi_range=True, extraStates=True)
print(f"Trained model (MODIFIED) with extra states for params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")

Trained model (NON-MODIFIED) with extra states for params [-1,1]: layers=13, fcnot_loss=0.518468423781077
Trained model (MODIFIED) with extra states for params [-1,1]: layers=13, fcnot_loss=0.5813965786336346
Trained model (NON-MODIFIED) with extra states for params [-pi,pi]: layers=13, fcnot_loss=0.5842887553791757
Trained model (MODIFIED) with extra states for params [-pi,pi]: layers=13, fcnot_loss=0.42624192671513017


In [14]:
l = 13
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=False, extraStates=False)
print(f"Trained model (NON-MODIFIED) with normal states for params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=True,extraStates=False)
print(f"Trained model (MODIFIED) with normal states for params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=False, pi_range=True, extraStates=False)
print(f"Trained model (NON-MODIFIED) with normal states for params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=True, pi_range=True, extraStates=False)
print(f"Trained model (MODIFIED) with normal states for params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")

Trained model (NON-MODIFIED) with normal states for params [-1,1]: layers=13, fcnot_loss=0.27665417339027903
Trained model (MODIFIED) with normal states for params [-1,1]: layers=13, fcnot_loss=0.3056212416029626
Trained model (NON-MODIFIED) with normal states for params [-pi,pi]: layers=13, fcnot_loss=0.47292994155429047
Trained model (MODIFIED) with normal states for params [-pi,pi]: layers=13, fcnot_loss=0.2950255204555837


In [26]:
l = 13
m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=False, batchOf4=1)
print(f"Trained model for first 4 states with params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=True, batchOf4=1)
print(f"Trained model for first 4 states with params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=False, batchOf4=2)
print(f"Trained model for second 4 states with params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=True, batchOf4=2)
print(f"Trained model for second 4 states with params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")

Trained model for first 4 states with params [-1,1]: layers=13, fcnot_loss=0.11289527726167026
Trained model for first 4 states with params [-pi,pi]: layers=13, fcnot_loss=0.13727806092228126
Trained model for second 4 states with params [-1,1]: layers=13, fcnot_loss=0.14017060530001393
Trained model for second 4 states with params [-pi,pi]: layers=13, fcnot_loss=0.13232137854327394


In [44]:
m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=False, batchOf4=1, modified=True)
print(f"Trained model for modified first 4 states with params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=True, batchOf4=1, modified=True)
print(f"Trained model for modified first 4 states with params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=False, batchOf4=2, modified=True)
print(f"Trained model for modified second 4 states with params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=True, batchOf4=2, modified=True)
print(f"Trained model for modified second 4 states with params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")

Trained model for modified first 4 states with params [-1,1]: layers=13, fcnot_loss=0.22975482460624957
Trained model for modified first 4 states with params [-pi,pi]: layers=13, fcnot_loss=0.19287227519506245
Trained model for modified second 4 states with params [-1,1]: layers=13, fcnot_loss=0.28040396252586397
Trained model for modified second 4 states with params [-pi,pi]: layers=13, fcnot_loss=0.18874626539730371


In [46]:
l = 7
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=False, extraStates=True)
print(f"Trained model (NON-MODIFIED) with extra states for params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=True, extraStates=True)
print(f"Trained model (MODIFIED) with extra states for params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=False, pi_range=True, extraStates=True)
print(f"Trained model (NON-MODIFIED) with extra states for params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=True, pi_range=True, extraStates=True)
print(f"Trained model (MODIFIED) with extra states for params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")

m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=False, extraStates=False)
print(f"Trained model (NON-MODIFIED) with normal states for params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=True,extraStates=False)
print(f"Trained model (MODIFIED) with normal states for params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=False, pi_range=True, extraStates=False)
print(f"Trained model (NON-MODIFIED) with normal states for params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=True, pi_range=True, extraStates=False)
print(f"Trained model (MODIFIED) with normal states for params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")

m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=False, batchOf4=1)
print(f"Trained model for first 4 states with params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=True, batchOf4=1)
print(f"Trained model for first 4 states with params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=False, batchOf4=2)
print(f"Trained model for second 4 states with params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=True, batchOf4=2)
print(f"Trained model for second 4 states with params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")

m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=False, batchOf4=1, modified=True)
print(f"Trained model for modified first 4 states with params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=True, batchOf4=1, modified=True)
print(f"Trained model for modified first 4 states with params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=False, batchOf4=2, modified=True)
print(f"Trained model for modified second 4 states with params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=True, batchOf4=2, modified=True)
print(f"Trained model for modified second 4 states with params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")

Trained model (NON-MODIFIED) with extra states for params [-1,1]: layers=7, fcnot_loss=0.8825795130608739
Trained model (MODIFIED) with extra states for params [-1,1]: layers=7, fcnot_loss=0.6700158300423998
Trained model (NON-MODIFIED) with extra states for params [-pi,pi]: layers=7, fcnot_loss=0.7597794087369221
Trained model (MODIFIED) with extra states for params [-pi,pi]: layers=7, fcnot_loss=0.9387374038187708
Trained model (NON-MODIFIED) with normal states for params [-1,1]: layers=7, fcnot_loss=0.5727536542728501
Trained model (MODIFIED) with normal states for params [-1,1]: layers=7, fcnot_loss=0.6570580915016189
Trained model (NON-MODIFIED) with normal states for params [-pi,pi]: layers=7, fcnot_loss=0.5804529364077136
Trained model (MODIFIED) with normal states for params [-pi,pi]: layers=7, fcnot_loss=1.0000000007696217
Trained model for first 4 states with params [-1,1]: layers=7, fcnot_loss=0.23543665341332728
Trained model for first 4 states with params [-pi,pi]: layers=

In [47]:
l = 18
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=False, extraStates=True)
print(f"Trained model (NON-MODIFIED) with extra states for params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=True, extraStates=True)
print(f"Trained model (MODIFIED) with extra states for params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=False, pi_range=True, extraStates=True)
print(f"Trained model (NON-MODIFIED) with extra states for params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=True, pi_range=True, extraStates=True)
print(f"Trained model (MODIFIED) with extra states for params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")

m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=False, extraStates=False)
print(f"Trained model (NON-MODIFIED) with normal states for params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=True,extraStates=False)
print(f"Trained model (MODIFIED) with normal states for params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=False, pi_range=True, extraStates=False)
print(f"Trained model (NON-MODIFIED) with normal states for params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, modified=True, pi_range=True, extraStates=False)
print(f"Trained model (MODIFIED) with normal states for params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")

m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=False, batchOf4=1)
print(f"Trained model for first 4 states with params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=True, batchOf4=1)
print(f"Trained model for first 4 states with params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=False, batchOf4=2)
print(f"Trained model for second 4 states with params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=True, batchOf4=2)
print(f"Trained model for second 4 states with params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")

m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=False, batchOf4=1, modified=True)
print(f"Trained model for modified first 4 states with params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=True, batchOf4=1, modified=True)
print(f"Trained model for modified first 4 states with params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=False, batchOf4=2, modified=True)
print(f"Trained model for modified second 4 states with params [-1,1]: layers={l}, fcnot_loss={f_loss}")
m, p, f_loss = get_optimal_half_matrix(num_layers=l, pi_range=True, batchOf4=2, modified=True)
print(f"Trained model for modified second 4 states with params [-pi,pi]: layers={l}, fcnot_loss={f_loss}")

Trained model (NON-MODIFIED) with extra states for params [-1,1]: layers=18, fcnot_loss=0.4165036183694318
Trained model (MODIFIED) with extra states for params [-1,1]: layers=18, fcnot_loss=0.36372719304585405
Trained model (NON-MODIFIED) with extra states for params [-pi,pi]: layers=18, fcnot_loss=0.48653835012375013
Trained model (MODIFIED) with extra states for params [-pi,pi]: layers=18, fcnot_loss=0.3676823674719307
Trained model (NON-MODIFIED) with normal states for params [-1,1]: layers=18, fcnot_loss=0.31558265583264755
Trained model (MODIFIED) with normal states for params [-1,1]: layers=18, fcnot_loss=0.2925393751889955
Trained model (NON-MODIFIED) with normal states for params [-pi,pi]: layers=18, fcnot_loss=0.3334449121934268
Trained model (MODIFIED) with normal states for params [-pi,pi]: layers=18, fcnot_loss=0.2949956528341103
Trained model for first 4 states with params [-1,1]: layers=18, fcnot_loss=0.09375015398923997
Trained model for first 4 states with params [-pi,

# Manual matrix multiplication of what the paper suggested

In [42]:
from utils import MatrixUtils,DFS

def getU_cnot(extraStates=False):

    p1 = np.arccos(-1/np.sqrt(3))/np.pi
    p2 = np.arcsin(1/np.sqrt(3))/np.pi


    def Id_n(n):
        assert n >= 0
        if n==0:
            return 1
        temp = I
        for i in range(n-1):
            temp = np.kron(temp, I)
        return temp

    matrixUtils = MatrixUtils(pi_range=True, extended_states=extraStates) # pi_range is true since the parameters are in the range of [-pi,pi]
    U_ex = lambda p : matrixUtils.U_ex(p)


    bounds = [[3,4],[2,5],[3,4],[2,5],[1,4],[2,5],[1,4],[2,5],[1,4],[2,5],[3,4],[2,5],[3,4]]
    operators = [U_ex(p1),
    np.kron(U_ex(1/2),U_ex(p2)),
    U_ex(1),
    np.kron(U_ex(-1/2),U_ex(-1/2)),
    np.kron(U_ex(1),U_ex(-1/2)),
    np.kron(U_ex(-1/2),U_ex(1)),
    np.kron(U_ex(-1/2),U_ex(1/2)),
    np.kron(U_ex(-1/2),U_ex(1)),
    np.kron(U_ex(1),U_ex(-1/2)),
    np.kron(U_ex(-1/2),U_ex(-1/2)),
    U_ex(1),
    np.kron(U_ex(1/2),U_ex(1-p2)),
    U_ex(-p1)]


    newOps = []
    for i, (start, end) in enumerate(bounds):
        temp = matrixUtils.nestedKron(Id_n(start),operators[i], Id_n(5-end))
        newOps.append(temp.copy())

    totalOperator = np.eye(2**6)
    for op in newOps:
        totalOperator = np.matmul(op,totalOperator)

    U_cnot = totalOperator.copy()
    return U_cnot

def applyOperator(operator, states):
    return [np.matmul(operator, mat) for mat in states]


def getLoss(extraStates=False, batchOf4=None, modified=False):
    if batchOf4 != None:
        extraStates = False
        modified=False

    matrixUtils = MatrixUtils(pi_range=True, extended_states=extraStates)

    originalInputStates, originalExpectedStates = DFS().getInitialTargetStates(extraStates=extraStates)
    modifiedInputStates, modifiedExpectedStates = DFS().getModifiedInitialTargetStates(extraStates=extraStates)
    if batchOf4 == 1:
        originalInputStates, originalExpectedStates = originalInputStates[:4], originalExpectedStates[:4]
        modifiedInputStates, modifiedExpectedStates = modifiedInputStates[:4], modifiedExpectedStates[:4]
    elif batchOf4 == 2:
        originalInputStates, originalExpectedStates = originalInputStates[4:], originalExpectedStates[4:]
        modifiedInputStates, modifiedExpectedStates = modifiedInputStates[4:], modifiedExpectedStates[4:]

    inputStates, expectedStates = (modifiedInputStates, modifiedExpectedStates) if modified else (originalInputStates, originalExpectedStates)

    loss_function = matrixUtils.f_cnot_loss if batchOf4==None else getLossFunctionOf4States

    U_cnot = getU_cnot(extraStates=extraStates)
    predStates = applyOperator(U_cnot, inputStates)
    print(f"For extraStates={extraStates}&batchOf4={batchOf4}&modified={modified}, then loss => {loss_function(expectedStates, predStates)}")

In [43]:
getLoss()

getLoss(extraStates=True)

getLoss(modified=True)

getLoss(modified=True, extraStates=True)

getLoss(batchOf4=1)

getLoss(batchOf4=2)


For extraStates=False&batchOf4=None&modified=False, then loss => 1.144257634055127
For extraStates=True&batchOf4=None&modified=False, then loss => 1.4300245571726808
For extraStates=False&batchOf4=None&modified=True, then loss => 1.2497750693297125
For extraStates=True&batchOf4=None&modified=True, then loss => 1.5224954010638858
For extraStates=False&batchOf4=1&modified=False, then loss => 0.683762932519148
For extraStates=False&batchOf4=2&modified=False, then loss => 0.9174932071717218
